# Import, and analyze audio files

In [1]:
import os
import statistics
import numpy as np
from matplotlib import pyplot as plt
import IPython.display as ipd
import librosa, librosa.display
import pandas as pd
import math
import ffmpeg
import pydub
import csv
import sklearn
import sounddevice as sd#to play back sounds
from pydub import AudioSegment
from scipy import signal#for fast correlation (using fast convolution)
from scipy.signal import butter, lfilter, freqz, sosfreqz, sosfilt
from scipy.signal import find_peaks
import scipy

In [4]:
# This is a band pass filter:
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    sos = butter(order, [low, high], btype='band',output='sos')
    return sos

def butter_bandpass_filter(data, lowcut, highcut, sr, order=5):
    sos = butter_bandpass(lowcut, highcut, sr, order=order)
    y = sosfilt(sos, data)
    return y

In [5]:
# Extraction  of all recording studio features per frame
def recordingstudio(cut, fs):
    bc = []
    peaks = []
    rmss = []
    crests = []
    corr=[]
    line=[]
    #klasse=[]
    for frame in range(int(len(cut[0])/fs)):
        box = [[0,0] for i in range(fs)]
        left = [0 for i in range(fs)]
        right = left
        for i in range(fs):
            left[i]=cut[0][frame*fs+i]
            right[i]=cut[1][frame*fs+i]
            box[i]=[int(math.floor(left[i]*20)),int(math.floor(right[i]*20))]
            sq = left[i]**2
        peak=np.max(np.abs(left))
        if peak < 0.000001:
            peak = 0.000001
        #print(peak)
        corr.append(np.min(np.corrcoef(cut[0][frame*fs:frame*fs+fs],cut[1][frame*fs:frame*fs+fs])))
        rms=np.sqrt(np.mean(sq))#0 to 1
        if rms < 0.0000002:
            rms = 0.0000002
        peakdB=20*math.log(peak,10)#-infty to 0
        rmsdB=20*math.log(rms,10)#0 to -infty
        crest=peakdB-rmsdB
        bc.append(len(np.unique(box))/400)#0.0025 to 1
        peaks.append(peakdB)
        rmss.append(rmsdB)
        crests.append(crest)#0 to 80.9
        line.append(frame)
        #klasse.append(1)
    #print('all frames done')
    return line, bc, corr, peaks, rmss, crests#,class 
#print(type(line))

In [6]:
# define source folder:
mp3list = []
path = r'C:\Users\t_ziemer\Desktop\techno-94\usa2025'#file path that contains all mp3 files to be analyzed
for root, dirs, files in os.walk(path):#loop to list all file paths
    for file in files:
        mp3list.append(os.path.join(root,file))
fs = 2048#frame size about 50 ms
ss = fs#step size equals frame size (i.e., no overlap between frames)

In [7]:
# loop through, export files and skip corrupted files
import warnings
# Suppress DeprecationWarning
warnings.simplefilter("ignore", category=DeprecationWarning)
for n in range(len(mp3list)):#len(mp3list)
    while True:
        try:
            #dur = pydub.utils.mediainfo(mp3list[n])["duration"]
            array, sr=librosa.load(mp3list[n],sr=44100,mono=False)#,duration = math.floor(float(dur))
            print(mp3list[n])
            #normalization:
            if np.max(array)>np.min(array)*-1:
                array = array/np.max(array)
            else:
                array = array/np.min(array)*-1
            cut = [array[0][176400:len(array[0])-176400],array[1][176400:len(array[1])-176400]]# we crop the first and last 4 seconds
            line, bc, corr, peak, rms, crest = recordingstudio(cut, fs)
            cutlow = [butter_bandpass_filter(cut[0],20, 150 ,sr, order=5),butter_bandpass_filter(cut[1],20, 150 ,sr, order=5)]
            line, bclow, corrlow, peaklow, rmslow, crestlow = recordingstudio(cutlow, fs)
            cutmid = [butter_bandpass_filter(cut[0],150, 2000 ,sr, order=5),butter_bandpass_filter(cut[1],150, 2000 ,sr, order=5)]
            line, bcmid, corrmid, peakmid, rmsmid, crestmid = recordingstudio(cutmid, fs)
            cuthigh = [butter_bandpass_filter(cut[0],2000, 10000 ,sr, order=5),butter_bandpass_filter(cut[1],2000, 10000 ,sr, order=5)]
            line, bchigh, corrhigh, peakhigh, rmshigh, cresthigh = recordingstudio(cuthigh, fs)
            tempo, beat_frames = librosa.beat.beat_track(y=cut[0], sr=sr)
            beats=[]            
            if tempo < 100:
                tempo = tempo * 2
            if tempo > 220:
                tempo = tempo / 2
            for i in range(len(line)):
                beats.append(tempo)#bpm,median
            #cutleft= cut[0][0::32]
            #estimated_bpm = estimate_bpm(cutleft)
            #median=np.median(estimated_bpm)
            features=[np.transpose(line), np.transpose(bc), np.transpose(corr), np.transpose(peak), np.transpose(rms), np.transpose(crest), np.transpose(bclow), np.transpose(corrlow), np.transpose(peaklow), np.transpose(rmslow), np.transpose(crestlow), np.transpose(bcmid), np.transpose(corrmid), np.transpose(peakmid), np.transpose(rmsmid), np.transpose(crestmid), np.transpose(bchigh), np.transpose(corrhigh), np.transpose(peakhigh), np.transpose(rmshigh), np.transpose(cresthigh),np.transpose(beats)] 
            Details = ["", 'PhaseSpace', 'ChannelCorrelation', 'PeakMeter', 'RMS', 'CrestFactor','PhaseSpaceLow', 'ChannelCorrelationLow', 'PeakMeterLow','RMSLow','CrestFactorLow', 'PhaseSpaceMid', 'ChannelCorrelationMid', 'PeakMeterMid','RMSMid','CrestFactorMid', 'PhaseSpaceHigh', 'ChannelCorrelationHigh', 'PeakMeterHigh','RMSHigh','CrestFactorHigh','bpm']
            with open(mp3list[n]+'.csv', 'w', newline='') as f:
                write = csv.writer(f) 
                write.writerow(Details) 
                write.writerows(np.transpose(features)) 
            break      
        except:
            print('shit!')
            break

C:\Users\t_ziemer\AppData\Local\Programs\Python\Python310\lib\site-packages\librosa\util\decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


C:\Users\t_ziemer\Desktop\techno-94\usa2025\A Bitch Named Johanna - Freak It (1990).m4a
C:\Users\t_ziemer\Desktop\techno-94\usa2025\A Bitch Named Johanna - Freak It (Club Mix) (1990).m4a
C:\Users\t_ziemer\Desktop\techno-94\usa2025\A Bitch Named Johanna - Freak It (Instrumental) (1990).opus


C:\Users\t_ziemer\AppData\Local\Programs\Python\Python310\lib\site-packages\librosa\util\decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


shit!
shit!
C:\Users\t_ziemer\Desktop\techno-94\usa2025\A Bitch Named Johanna - I'm A Bitch (X Bitch) (1990).m4a


C:\Users\t_ziemer\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\lib\function_base.py:2853: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]


...

C:\Users\t_ziemer\Desktop\techno-94\usa2025\Funky Green Dogs From Outer Space - Reach For Me (Long Ass Mix) (1992).m4a
C:\Users\t_ziemer\Desktop\techno-94\usa2025\Funky Green Dogs From Outer Space - Reach For Me (Vibe Mix) (1992).m4a
C:\Users\t_ziemer\Desktop\techno-94\usa2025\Gangsters Of Techno - Lost In A 'E' (1991).m4a
C:\Users\t_ziemer\Desktop\techno-94\usa2025\Gangsters Of Techno - Mind Control (Reactive Mix) (1991).m4a
C:\Users\t_ziemer\Desktop\techno-94\usa2025\Hammer - Gaining Momentum (Giant Club Mix) (1991).m4a
C:\Users\t_ziemer\Desktop\techno-94\usa2025\Hashim - Al Naafiysh (Boot That Remix) (1994).m4a
shit!
C:\Users\t_ziemer\Desktop\techno-94\usa2025\Housecat - Those Were The Dayz (Dance Mix) (1993).m4a
C:\Users\t_ziemer\Desktop\techno-94\usa2025\Housecat - Those Were The Dayz (Heartache Mix) (1993).m4a
C:\Users\t_ziemer\Desktop\techno-94\usa2025\Housecat - Those Were The Dayz (Underground Mix) (1993).m4a
C:\Users\t_ziemer\Desktop\techno-94\usa2025\Inner CIty - Faith (1992

C:\Users\t_ziemer\Desktop\techno-94\usa2025\Mike Dearborn - Outer Limits (Trance Mixx) (1993).m4a
shit!
shit!
C:\Users\t_ziemer\Desktop\techno-94\usa2025\Mike Dearborn - Sexual X-Perience (Original Mix) (1991).m4a
shit!
C:\Users\t_ziemer\Desktop\techno-94\usa2025\Mike Dearborn - Simply Complex (1992).m4a
C:\Users\t_ziemer\Desktop\techno-94\usa2025\Model 500 - No UFO's (D-Mix) (1985).m4a
C:\Users\t_ziemer\Desktop\techno-94\usa2025\MURK In Association With The Big Bang - Bugged Out V2.0 (Deep Double Acid) (1993).m4a
C:\Users\t_ziemer\Desktop\techno-94\usa2025\Nelson Rosado - Don't Try To Stop Me (1994).m4a
C:\Users\t_ziemer\Desktop\techno-94\usa2025\Nelson Rosado - Feel It (1994).m4a
C:\Users\t_ziemer\Desktop\techno-94\usa2025\Nelson Rosado - I Said Shut Up (1994).m4a
C:\Users\t_ziemer\Desktop\techno-94\usa2025\Nelson Rosado - I Said Shut Up (Bonus 'Bump' Beats) (1994).m4a
shit!
C:\Users\t_ziemer\Desktop\techno-94\usa2025\Paradise Presents Afrikali - Out Of The Jungle (The Truth Mix) (19

C:\Users\t_ziemer\Desktop\techno-94\usa2025\Roy Davis Jr. - Mind Power (DJ Pierres Original Wildpich Mix) (1993).m4a
C:\Users\t_ziemer\Desktop\techno-94\usa2025\Roy Davis Jr. - Mind Power (Roys Powerful Blunt Mix) (1993).m4a
C:\Users\t_ziemer\Desktop\techno-94\usa2025\Roy Davis Presents The Believers - Essence Of Life (3 A.M. Club Dub Mix) (1994).m4a
C:\Users\t_ziemer\Desktop\techno-94\usa2025\Roy Davis Presents The Believers - Essence Of Life (Bag-UV-Funk Mix) (1994).m4a
C:\Users\t_ziemer\Desktop\techno-94\usa2025\Roy Davis Presents The Believers - Essence Of Life (House Cat Radical Fear Mix) (1994).m4a
C:\Users\t_ziemer\Desktop\techno-94\usa2025\Roy Davis Presents The Believers - Essence Of Life (Roy's Hard Doors Mix) (1994).m4a
C:\Users\t_ziemer\Desktop\techno-94\usa2025\Roy Davis Presents The Believers - Essence Of Life (Stompin' Darkside Mix) (1994).m4a
C:\Users\t_ziemer\Desktop\techno-94\usa2025\Roy Davis Project - I Got The Music (Tha Musical Dub) (1993).m4a
C:\Users\t_ziemer\De

C:\Users\t_ziemer\Desktop\techno-94\usa2025\The Reese Project - So Deep (Bump Dub) (1993).m4a
C:\Users\t_ziemer\Desktop\techno-94\usa2025\The Reese Project - So Deep (CJ Mackintosh FXTC Club Anthem) (1993).m4a
C:\Users\t_ziemer\Desktop\techno-94\usa2025\The Reese Project - So Deep (Full Bump Mix) (1993).m4a
C:\Users\t_ziemer\Desktop\techno-94\usa2025\The Reese Project - So Deep (Joey Negro Direct Mix) (1993).m4a
C:\Users\t_ziemer\Desktop\techno-94\usa2025\The Reese Project - So Deep (Joey Negro K.O.K Club Mix) (1993).m4a
C:\Users\t_ziemer\Desktop\techno-94\usa2025\The Reese Project - Station Of The Groove (Jay Denham Mix) (1991).m4a
C:\Users\t_ziemer\Desktop\techno-94\usa2025\The Reese Project - The Colour Of Love (Deep Reese Mix) (1992).m4a
C:\Users\t_ziemer\Desktop\techno-94\usa2025\The Reese Project - The Colour Of Love (Groove Corporation Trance Mix) (1992).m4a
C:\Users\t_ziemer\Desktop\techno-94\usa2025\The Reese Project - The Colour Of Love (Groove Corporation Wobble Dub) (1992).

C:\Users\t_ziemer\Desktop\techno-94\usa2025\Triple XXX - The Bedroom Scene (Straight Up Mix) (1987).m4a
C:\Users\t_ziemer\Desktop\techno-94\usa2025\Triple XXX - The Bedroom Scene (Waterbed Mix) (1987).opus
C:\Users\t_ziemer\Desktop\techno-94\usa2025\Tropical Moon - Data Bank (1990).m4a
C:\Users\t_ziemer\Desktop\techno-94\usa2025\Tropical Moon - It's Alright (1990).m4a
C:\Users\t_ziemer\Desktop\techno-94\usa2025\Tropical Moon - Jungle House (1990).m4a
C:\Users\t_ziemer\Desktop\techno-94\usa2025\Tropical Moon - Love Is A Mystery (1990).m4a
C:\Users\t_ziemer\Desktop\techno-94\usa2025\Troy Abstract - Talkin' Shit (Chunk-A-Bud's Generation X Mix) (1994).opus
C:\Users\t_ziemer\Desktop\techno-94\usa2025\Troy Abstract - Talkin' Shit (Dub Fiasco Mix) (1994).m4a
C:\Users\t_ziemer\Desktop\techno-94\usa2025\Troy Abstract - Talkin' Shit (Trance Mix) (1994).opus
C:\Users\t_ziemer\Desktop\techno-94\usa2025\Troy Abstract - Talkin' Shit (Tribal Trance Mix) (1994).m4a
C:\Users\t_ziemer\Desktop\techno-94